## Set Global Seed 

In [1]:
import os
import random
import numpy as np
import torch
import transformers

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed = 260615
set_all_seeds(seed)

print("The global seed " + str(seed))

The global seed 260615


## Hyperparameters

In [2]:
# LANGUAGE

_LANGUAGE_         = 'es'

In [3]:
# MODEL CLASSIFICATION

_PRETRAINED_LM_    = 'pysentimiento/robertuito-base-uncased'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig(reduction_factor = 256)
_MAX_SEQ_LEN_      = 128

In [4]:
# TRAIN

_OUTPUT_DIR_       = 'checkPointsFFT'
_LOGGING_STEPS_    = 50
_NUM_AUTHORS_      = 8
_K_FOLD_CV_        = 5
_NO_GPUS_          = 1
_BATCH_SIZE_       = int(32 / _NO_GPUS_)
_EPOCHS_           = 10
_LEARNING_RATE_    = 1e-5

# PREDICTIONS

_DATASET_          = 'PAN17'
_PRED_DIR_         = 'FFT_5tweet'

## Other parameters

In [5]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 'male':   1}
varietyEN_dict = {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}
varietyES_dict = {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}  

genderEN_hip  = {0: 'I’m a female', 1: 'I’m a male'}
genderES_hip  = {0: 'Mi nombre es María', 1: 'Mi nombre es José'}

In [6]:
# SET LANGUAGE DICTIONARIES

if _LANGUAGE_ == 'en':
    gender_hip   = genderEN_hip
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    gender_hip   = genderES_hip
    variety_dict = varietyES_dict

In [7]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer, PretrainedConfig

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

## Datasets

In [8]:
from tools.DataLoaders import BasePAN17

baseTrain  = BasePAN17(Dir             = 'data/2017',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = 1,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir              = 'data/2017',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = 1,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 419998


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 280000



In [9]:
crossVal_splits = []

for val_idx in range(_K_FOLD_CV_):
    
    authors_train, authors_val = baseTrain.cross_val(_K_FOLD_CV_, val_idx, _NUM_AUTHORS_)
    
    crossVal_splits.append( (authors_train, authors_val) )

In [10]:
from tools.DataLoaders import BasePAN17

baseTrain  = BasePAN17(Dir             = 'data/2017',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir              = 'data/2017',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 56000



In [11]:
from tools.DataLoaders import DatasetPAN17

Test = DatasetPAN17(baseTest, 'gender')

In [12]:
crossVal_splits[0][0]

['bafc7670beae172539e66c48858d8a',
 '70250236e5aae9df16d3d081b24f8c52',
 '4230dfe217cc7d667e6a21bff301263d',
 'b1be8bd39d7a3be53d2e37a654db05d8',
 '8690c40c4fea5fdb6ad1c030cafca7f',
 'eb3d30f6a613d63c495faf276ce8af20',
 'aeef24da173bad8c8589593068d12e2b',
 '3fccf54ed8ff8ebf3e09b27178c3eda6',
 'bca70cfd674db9f5e0885728d43043e7',
 'cdca31bfd55d33ef52918fbd1b4e16cc',
 '4e0f3dc3fa3a35480d7a27bf12816ae1',
 'bad426ef9226de53f904ee17f813abb1',
 '921ec4a999f5b8c26f2102e6a6fd2e26',
 '6b8c65a63a47a179aa0729944df6c16d',
 'df3eaaf1eadbc7de0f8c496a4d5adf0b',
 'a216de7f51f7022889e2eb86e48155d8']

## Training

In [13]:
from transformers import TrainingArguments

samples = 2 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 10,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [14]:
from transformers import AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossVal
from transformers import Trainer
from tools.Testing import compute_author_predictions
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = 'gender'

f1s_soft = []
f1s_hard = []

for split in range( _K_FOLD_CV_ ):
    
    # loaders for current split ------------------------------------------
    
    authors_train, authors_val = crossVal_splits[split]
    
    Train = DatasetCrossVal(baseTrain, authors_train, task)
    Val   = DatasetCrossVal(baseTrain, authors_val  , task)
    
    
    # initialize model ---------------------------------------------------
    
    model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_, num_labels = 2)
    
    
    # create trainer and train -------------------------------------------
        
    trainer = Trainer(
        model           = model,
        args            = training_args,
        train_dataset   = Train,
    )
    trainer.args._n_gpu = _NO_GPUS_

    trainer.train()
    
    
    # get predictions ----------------------------------------------------
    
    results            = trainer.predict(Val)
    author_predictions = compute_author_predictions(Val, results.predictions, 'gender', 2)
    
    
    # report metrics -----------------------------------------------------
    
    report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

    f1s_soft.append( f1_score(author_predictions['true'], author_predictions['pred_soft'], average = 'macro') )
    f1s_hard.append( f1_score(author_predictions['true'], author_predictions['pred_hard'], average = 'macro') )

    print("Results with split " + str(split + 1) + ":\n")
    print("soft voting:\n", report['soft'], '\n')
    print("hard voting:\n", report['hard'])
     
    
    # save predictions ----------------------------------------------------
    
    DIR = 'results/' + _DATASET_ + '/' + _LANGUAGE_ + '/' + _PRED_DIR_ + '/' + str(_NUM_AUTHORS_) + '_authors/split_' + str(split + 1) + '/'
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    with open(DIR + 'predictions.pickle', 'wb') as f:
        pickle.dump(author_predictions, f)

    with open(DIR + 'report.txt', 'w') as f:
        f.write("soft voting:\n" + report['soft'] + '\n\n')
        f.write("hard voting:\n" + report['hard'])
    


Some weights of the model checkpoint at pysentimiento/robertuito-base-uncased were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.

Step,Training Loss
10,0.653600
20,0.527200
30,0.422700
40,0.343900
50,0.285200
60,0.230000
70,0.207500
80,0.173500
90,0.157600
100,0.151600




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 16800
  Batch size = 200


100%|██████████████████████████████████████████████████████| 840/840 [00:04<00:00, 199.13it/s]


Results with split 1:

soft voting:
               precision    recall  f1-score   support

           0     0.6489    0.5500    0.5954       420
           1     0.6095    0.7024    0.6527       420

    accuracy                         0.6262       840
   macro avg     0.6292    0.6262    0.6240       840
weighted avg     0.6292    0.6262    0.6240       840
 

hard voting:
               precision    recall  f1-score   support

           0     0.6364    0.5500    0.5900       420
           1     0.6038    0.6857    0.6421       420

    accuracy                         0.6179       840
   macro avg     0.6201    0.6179    0.6161       840
weighted avg     0.6201    0.6179    0.6161       840



...

loading configuration file https://huggingface.co/pysentimiento/robertuito-base-uncased/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/5212cb9b5b32726fce956daa9a21ee0a0c2b6e54c54d1af58c678217d85f8143.4cce50d5a926bf18fe43f2ea8d4596b505e97a64e6e700e993def66b06f1c83b
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-base-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "

Step,Training Loss
10,0.657800
20,0.560900
30,0.481800
40,0.421900
50,0.373700
60,0.338300
70,0.300700
80,0.278700
90,0.260800
100,0.257900




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 16800
  Batch size = 200


100%|██████████████████████████████████████████████████████| 840/840 [00:04<00:00, 191.00it/s]

Results with split 5:

soft voting:
               precision    recall  f1-score   support

           0     0.5964    0.7143    0.6501       420
           1     0.6439    0.5167    0.5733       420

    accuracy                         0.6155       840
   macro avg     0.6202    0.6155    0.6117       840
weighted avg     0.6202    0.6155    0.6117       840
 

hard voting:
               precision    recall  f1-score   support

           0     0.5872    0.7214    0.6474       420
           1     0.6389    0.4929    0.5565       420

    accuracy                         0.6071       840
   macro avg     0.6130    0.6071    0.6019       840
weighted avg     0.6130    0.6071    0.6019       840



In [15]:
# report statistics

print('Soft results: ', f1s_soft)
print('\nHard results: ', f1s_hard)

f1s_soft = np.array(f1s_soft)
f1s_hard = np.array(f1s_hard)

FewShot_Results = {'soft': [f1s_soft.mean(), f1s_soft.std()], 'hard': [f1s_hard.mean(), f1s_hard.std()]}

print('\n\nSoft statistics: ')
print('\t[avg, std]:', FewShot_Results['soft'])

print('\nHard statistics: ')
print('\t[avg, std]:', FewShot_Results['hard'])

Soft results:  [0.6240078459994526, 0.5318536999497548, 0.5290617848970252, 0.5652994356097645, 0.6116849455640458]

Hard results:  [0.616089391201835, 0.5360833685135815, 0.5333333333333333, 0.5653588541435244, 0.6019437551695617]


Soft statistics: 
	[avg, std]: [0.5723815424040086, 0.03944438201821679]

Hard statistics: 
	[avg, std]: [0.5705617404723672, 0.033643365719226874]


## Test the best split

In [16]:
from transformers import AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossVal, DatasetPAN17
from transformers import Trainer
from tools.Testing import compute_author_predictions
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = 'gender'

split = 1

# loaders for current split ------------------------------------------
    
authors_train, authors_val = crossVal_splits[split]

Train = DatasetCrossVal(baseTrain, authors_train, task)
Val   = DatasetCrossVal(baseTrain, authors_val  , task)


# initialize model ---------------------------------------------------
    
model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_, num_labels = 2)


# create trainer and train -------------------------------------------

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = Train,
)
trainer.args._n_gpu = _NO_GPUS_

trainer.train()


# get predictions ----------------------------------------------------

results            = trainer.predict(Test)
author_predictions = compute_author_predictions(baseTest, results.predictions, 'gender', 2)


# report metrics -----------------------------------------------------

report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
           'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

print("Results with split " + str(split + 1) + ":\n")
print("soft voting:\n", report['soft'], '\n')
print("hard voting:\n", report['hard'])


loading configuration file https://huggingface.co/pysentimiento/robertuito-base-uncased/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/5212cb9b5b32726fce956daa9a21ee0a0c2b6e54c54d1af58c678217d85f8143.4cce50d5a926bf18fe43f2ea8d4596b505e97a64e6e700e993def66b06f1c83b
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-base-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "

Step,Training Loss
160,0.619600
320,0.496200
480,0.396200
640,0.299000
800,0.220000
960,0.152800
1120,0.105700
1280,0.079200
1440,0.059600
1600,0.047700


Saving model checkpoint to checkPointsFFT/checkpoint-500
Configuration saved in checkPointsFFT/checkpoint-500/config.json
Model weights saved in checkPointsFFT/checkpoint-500/pytorch_model.bin
Saving model checkpoint to checkPointsFFT/checkpoint-1000
Configuration saved in checkPointsFFT/checkpoint-1000/config.json
Model weights saved in checkPointsFFT/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to checkPointsFFT/checkpoint-1500
Configuration saved in checkPointsFFT/checkpoint-1500/config.json
Model weights saved in checkPointsFFT/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 56000
  Batch size = 200


100%|█████████████████████████████████████████████████████| 2800/2800 [00:31<00:00, 89.07it/s]

Results with split 2:

soft voting:
               precision    recall  f1-score   support

           0     0.7855    0.7507    0.7677      1400
           1     0.7613    0.7950    0.7778      1400

    accuracy                         0.7729      2800
   macro avg     0.7734    0.7729    0.7727      2800
weighted avg     0.7734    0.7729    0.7727      2800
 

hard voting:
               precision    recall  f1-score   support

           0     0.7639    0.7650    0.7645      1400
           1     0.7647    0.7636    0.7641      1400

    accuracy                         0.7643      2800
   macro avg     0.7643    0.7643    0.7643      2800
weighted avg     0.7643    0.7643    0.7643      2800

